# Рекомендательная система
## Обучение контрольной модели

**Описание задачи**

На данном шаге необходимо обучить модель с применением классических методов машинного обучения.

**Описание данных**

Датасет `user_data` cодержит информацию о всех пользователях социальной сети.

- age — возраст пользователя (в профиле),
- city — город пользователя (в профиле),
- country — страна пользователя (в профиле),
- exp_group — экспериментальная группа (зашифрованная категория),
- gender — пол пользователя,
- user_id — уникальный идентификатор пользователя,
- os — операционная система устройства, с которого происходит пользование соц.етью,
- source — признак, показывающий пришел ли пользователь в приложение с органического трафика или с рекламы.

Датасет `post_text_df` содержит информацию о постах и уникальный ID каждой единицы с соответствующим ей текстом и топиком.

- id — уникальный идентификатор поста,
- text — текстовое содержание поста,
- topic — основная тематика.

Датасет `feed_data` содержит историю о просмотренных постах для каждого юзера в изучаемый период.

- timestamp — время, когда был произведен просмотр,
- user_id — уникальный идентификатор пользователя, который совершил просмотр,
- post_id — уникальный идентификатор просмотренного поста,
- action — тип действия (просмотр или лайк),
- target — 1 у просмотров, если почти сразу после просмотра был совершен лайк, иначе 0; у действий like пропущенное значение.

**Последовательность выполнения**

- Загрузить датасеты;
- Провести кодировку признаков (бинарную, One Hot Encoding, TF-IDF);
- Подготовить датасеты для обучения модели;
- Разделить выборки на обучающую и тестовую;
- Обучить модель и замерить качество;
- Сохранить модель.

Загрузим библиотеки.

In [7]:
import pandas as pd
from sqlalchemy import create_engine

Создадим подключения к базе данных PostgreSQL.

In [10]:
# параметры подключения удалены
engine = create_engine(
   
)

Загрузим датасет `user_data`.

In [13]:
user_data = pd.read_sql('SELECT * FROM public.user_data', con=engine, params=None)
user_data

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads
...,...,...,...,...,...,...,...,...
163200,168548,0,36,Russia,Kaliningrad,4,Android,organic
163201,168549,0,18,Russia,Tula,2,Android,organic
163202,168550,1,41,Russia,Yekaterinburg,4,Android,organic
163203,168551,0,38,Russia,Moscow,3,iOS,organic


Проведем кодировку признаков `os` и `source` (в каждой группе по 2 значения) бинарным кодированием. Для признака `country` (11 значений) применим One Hot Encoding. Признак `city` (3915 значений) оставим как категориальный признак.

In [17]:
user_data['os'] = user_data['os'].map({'Android': 0, 'iOS': 1})
user_data['source'] = user_data['source'].map({'ads': 0, 'organic': 1})

In [19]:
encoded_columns = pd.get_dummies(user_data['country'], prefix='country', drop_first=True).astype(int)
user_data = pd.concat([user_data, encoded_columns], axis=1)
user_data.drop('country', axis=1, inplace=True)

In [21]:
user_data

,user_id,gender,age,city,exp_group,os,source,country_Belarus,country_Cyprus,country_Estonia,country_Finland,country_Kazakhstan,country_Latvia,country_Russia,country_Switzerland,country_Turkey,country_Ukraine
0,200,1,34,Degtyarsk,3,0,0,0,0,0,0,0,0,1,0,0,0
1,201,0,37,Abakan,0,0,0,0,0,0,0,0,0,1,0,0,0
2,202,1,17,Smolensk,4,0,0,0,0,0,0,0,0,1,0,0,0
3,203,0,18,Moscow,1,1,0,0,0,0,0,0,0,1,0,0,0
4,204,0,36,Anzhero-Sudzhensk,3,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163200,168548,0,36,Kaliningrad,4,0,1,0,0,0,0,0,0,1,0,0,0
163201,168549,0,18,Tula,2,0,1,0,0,0,0,0,0,1,0,0,0
163202,168550,1,41,Yekaterinburg,4,0,1,0,0,0,0,0,0,1,0,0,0
163203,168551,0,38,Moscow,3,1,1,0,0,0,0,0,0,1,0,0,0


Загрузим датасет `post_text_df`.

In [26]:
engine.dispose()
post_text_df = pd.read_sql('SELECT * FROM public.post_text_df', con=engine, params=None)
post_text_df

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business
...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie
7019,7316,I give this movie 2 stars purely because of it...,movie
7020,7317,I cant believe this film was allowed to be mad...,movie
7021,7318,The version I saw of this film was the Blockbu...,movie


Для признака `topic` (7 значений) применим One Hot Encoding.

In [29]:
encoded_columns = pd.get_dummies(post_text_df['topic'], prefix='topic', drop_first=True).astype(int)
post_text_df = pd.concat([post_text_df, encoded_columns], axis=1)
post_text_df.drop('topic', axis=1, inplace=True)

Для признака 'text' применим кодировку TF-IDF.

In [32]:
import re
import string
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [34]:
wnl = WordNetLemmatizer()

def preprocessing(line, token=wnl):
    line = line.lower()
    line = re.sub(r"[{}]".format(string.punctuation), " ", line)
    line = line.replace('\n\n', ' ').replace('\n', ' ')
    line = ' '.join([token.lemmatize(x) for x in line.split(' ')])
    return line


tfidf = TfidfVectorizer(
    stop_words='english',
    preprocessor=preprocessing
)

In [36]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\EliteBook\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [38]:
tfidf_data = (
    tfidf
    .fit_transform(post_text_df['text'])
    .toarray()
)

C:\Users\EliteBook\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'wa'] not in stop_words.
  warnings.warn(


In [40]:
tfidf_data = pd.DataFrame(
    tfidf_data,
    index=post_text_df.post_id,
    columns=tfidf.get_feature_names_out()
)

In [42]:
post_text_df['TotalTfIdf'] = tfidf_data.sum(axis=1).reset_index()[0]
post_text_df['MaxTfIdf'] = tfidf_data.max(axis=1).reset_index()[0]
post_text_df['MeanTfIdf'] = tfidf_data.mean(axis=1).reset_index()[0]

In [44]:
post_text_df.drop('text', axis=1, inplace=True)

In [46]:
post_text_df

,post_id,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech,TotalTfIdf,MaxTfIdf,MeanTfIdf
0,1,0,0,0,0,0,0,8.748129,0.495805,0.000190
1,2,0,0,0,0,0,0,11.878472,0.308003,0.000258
2,3,0,0,0,0,0,0,12.675530,0.261799,0.000275
3,4,0,0,0,0,0,0,6.622786,0.537713,0.000144
4,5,0,0,0,0,0,0,6.352096,0.420251,0.000138
...,...,...,...,...,...,...,...,...,...,...
7018,7315,0,0,1,0,0,0,6.703322,0.258411,0.000146
7019,7316,0,0,1,0,0,0,6.471398,0.399418,0.000141
7020,7317,0,0,1,0,0,0,5.628524,0.573061,0.000122
7021,7318,0,0,1,0,0,0,6.883375,0.231244,0.000150


Загрузим 5000000 строк из датасета `feed_data`.

In [50]:
engine.dispose()
feed_data = pd.read_sql('SELECT * FROM public.feed_data  order by timestamp limit 5000000', con=engine, params=None)
feed_data

,timestamp,user_id,post_id,action,target
0,2021-10-01 06:01:40,29084,2845,view,1
1,2021-10-01 06:01:40,156604,573,view,1
2,2021-10-01 06:01:40,22276,5717,view,1
3,2021-10-01 06:01:40,1859,1498,view,1
4,2021-10-01 06:01:40,87173,6030,view,0
...,...,...,...,...,...
4999995,2021-10-06 22:59:06,140970,6289,view,0
4999996,2021-10-06 22:59:06,134168,321,view,0
4999997,2021-10-06 22:59:06,99060,6650,view,0
4999998,2021-10-06 22:59:06,20246,5702,view,0


Удалим строки с лайками, удалим столбец `action`.

In [53]:
feed_data = feed_data[feed_data['action'] != 'like']
feed_data.drop('action', axis=1, inplace=True)

C:\Users\EliteBook\AppData\Local\Temp\ipykernel_11556\465158334.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feed_data.drop('action', axis=1, inplace=True)


Создадим временные признаки - выделим из времени совершённого действия месяц, день недели и час.

In [56]:
feed_data["timestamp"] = pd.to_datetime(feed_data["timestamp"])

feed_data['month'] = feed_data["timestamp"].dt.month.astype(int)
feed_data['hour'] = feed_data["timestamp"].dt.hour.astype(int)
feed_data['weekday'] = feed_data["timestamp"].dt.weekday.astype(int)

feed_data.drop('timestamp', axis=1, inplace=True)

C:\Users\EliteBook\AppData\Local\Temp\ipykernel_11556\785391618.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feed_data["timestamp"] = pd.to_datetime(feed_data["timestamp"])
C:\Users\EliteBook\AppData\Local\Temp\ipykernel_11556\785391618.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feed_data['month'] = feed_data["timestamp"].dt.month.astype(int)
C:\Users\EliteBook\AppData\Local\Temp\ipykernel_11556\785391618.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [58]:
feed_data

,user_id,post_id,target,month,hour,weekday
0,29084,2845,1,10,6,4
1,156604,573,1,10,6,4
2,22276,5717,1,10,6,4
3,1859,1498,1,10,6,4
4,87173,6030,0,10,6,4
...,...,...,...,...,...,...
4999995,140970,6289,0,10,22,2
4999996,134168,321,0,10,22,2
4999997,99060,6650,0,10,22,2
4999998,20246,5702,0,10,22,2


Объединим все таблицы в одну, удалим индексы.

In [61]:
merged_df = pd.merge(feed_data, user_data, on='user_id', how='inner')
df = pd.merge(merged_df, post_text_df, on='post_id', how='inner')

In [63]:
df.drop(['user_id', 'post_id'], axis=1, inplace=True)

In [65]:
df

,target,month,hour,weekday,gender,age,city,exp_group,os,source,...,country_Ukraine,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech,TotalTfIdf,MaxTfIdf,MeanTfIdf
0,1,10,6,4,1,15,Mikhaylovka,4,0,0,...,0,1,0,0,0,0,0,3.296298,0.376926,0.000072
1,1,10,6,4,1,30,Chernihiv,1,0,1,...,1,0,1,0,0,0,0,8.718607,0.363419,0.000189
2,1,10,6,4,0,30,Cheboksary,1,1,0,...,0,0,0,1,0,0,0,5.736243,0.362383,0.000125
3,1,10,6,4,0,19,Kyiv,3,0,0,...,1,0,0,0,0,1,0,12.430876,0.294456,0.000270
4,0,10,6,4,1,36,Novokuznetsk,0,0,0,...,0,0,0,1,0,0,0,8.313837,0.255433,0.000181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4576277,0,10,22,2,0,49,Brest,2,1,1,...,0,0,0,1,0,0,0,6.423904,0.353426,0.000140
4576278,0,10,22,2,0,25,Moscow,2,1,1,...,0,0,0,0,0,0,0,9.025208,0.410648,0.000196
4576279,0,10,22,2,1,25,Belovo,1,0,0,...,0,0,0,1,0,0,0,5.861137,0.437757,0.000127
4576280,0,10,22,2,0,58,Proletarsk,0,1,0,...,0,0,0,1,0,0,0,7.633536,0.650222,0.000166


Разделим данные на признаки и целевую переменную, выделим обучающую и тестовую выборки.

In [68]:
X = df.drop('target', axis=1)
y = df['target']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

Обучим модель с помощью библиотеки CatBoost.

In [71]:
!pip install catboost

   ---------------------------------------- 0.0/102.4 MB ? eta -:--:--
   ---------------------------------------- 0.3/102.4 MB ? eta -:--:--
   ---------------------------------------- 0.5/102.4 MB 1.7 MB/s eta 0:01:01
   ---------------------------------------- 0.8/102.4 MB 1.6 MB/s eta 0:01:04
   ---------------------------------------- 1.0/102.4 MB 1.6 MB/s eta 0:01:03
    --------------------------------------- 1.6/102.4 MB 1.6 MB/s eta 0:01:04
    --------------------------------------- 1.8/102.4 MB 1.6 MB/s eta 0:01:02
    --------------------------------------- 2.1/102.4 MB 1.6 MB/s eta 0:01:02
   - -------------------------------------- 2.6/102.4 MB 1.7 MB/s eta 0:01:01
   - -------------------------------------- 2.9/102.4 MB 1.7 MB/s eta 0:01:00
   - -------------------------------------- 3.4/102.4 MB 1.7 MB/s eta 0:01:00
   - -------------------------------------- 3.7/102.4 MB 1.7 MB/s eta 0:00:59
   - -------------------------------------- 3.9/102.4 MB 1.7 MB/s eta 0:01:00


In [73]:
from catboost import CatBoostClassifier

catboost = CatBoostClassifier(iterations=500,
                              learning_rate=1,
                              depth=2,
                              random_seed=100,
                              verbose=10,
                              cat_features=['city'])

catboost.fit(X_train, y_train)

0:	learn: 0.3085046	total: 2.37s	remaining: 19m 43s
10:	learn: 0.2965946	total: 16.5s	remaining: 12m 14s
20:	learn: 0.2925216	total: 28.5s	remaining: 10m 50s
30:	learn: 0.2921791	total: 41.3s	remaining: 10m 25s
40:	learn: 0.2920066	total: 53.5s	remaining: 9m 58s
50:	learn: 0.2918274	total: 1m 4s	remaining: 9m 29s
60:	learn: 0.2916227	total: 1m 16s	remaining: 9m 10s
70:	learn: 0.2915782	total: 1m 27s	remaining: 8m 46s
80:	learn: 0.2915089	total: 1m 39s	remaining: 8m 34s
90:	learn: 0.2914337	total: 1m 49s	remaining: 8m 13s
100:	learn: 0.2912725	total: 2m 1s	remaining: 8m
110:	learn: 0.2911976	total: 2m 13s	remaining: 7m 46s
120:	learn: 0.2911363	total: 2m 25s	remaining: 7m 34s
130:	learn: 0.2911143	total: 2m 35s	remaining: 7m 18s
140:	learn: 0.2910448	total: 2m 47s	remaining: 7m 6s
150:	learn: 0.2909932	total: 2m 58s	remaining: 6m 53s
160:	learn: 0.2909643	total: 3m 11s	remaining: 6m 43s
170:	learn: 0.2909202	total: 3m 23s	remaining: 6m 31s
180:	learn: 0.2908540	total: 3m 35s	remaining: 

Оценим качество.

In [77]:
from sklearn.metrics import roc_auc_score

print(f"Качество на трейне: {roc_auc_score(y_train, catboost.predict_proba(X_train)[:, 1])}")
print(f"Качество на тесте: {roc_auc_score(y_test, catboost.predict_proba(X_test)[:, 1])}")

Качество на трейне: 0.6912042334688595
Качество на тесте: 0.6860399039995173


Сохраним модель.

In [79]:
catboost.save_model('control_model', format="cbm")

Сохраним таблицы с признаками постов и юзеров для обученной модели.

In [36]:
post_text_df.to_sql('l_22_posts_features', con=engine, if_exists='replace', index=False)

23

In [39]:
user_data.to_sql('l_22_users_features', con=engine, if_exists='replace', index=False)

205